# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
# 1) Seu código aqui
# URL da API com filtros para Municípios (interessado_tipo) e Rio de Janeiro (uf)
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio"

# Requisição dos dados
r = requests.get(url)
data_json = r.json()

# Convertendo para DataFrame (os dados ficam na chave 'items')
df = pd.DataFrame(data_json['items'])

# Visualizando as primeiras linhas
df.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,6981,Município,Rio de Janeiro,3304557,RJ,None,Deferido,17944.000472/2007-90,2007-04-30T00:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,7.403640e+06,1,0,25/05/2007
1,13241,Município,Barra Mansa,3300407,RJ,None,Deferido,17944.001974/2006-57,2007-06-11T00:00:00Z,Operação contratual interna,Provias,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1.980000e+06,1,0,15/06/2007
2,13440,Município,Rio de Janeiro,3304557,RJ,None,Deferido,19405.000001/2004-61,2004-02-26T00:00:00Z,Operação contratual interna,PMAT,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.042000e+07,1,0,04/03/2004
3,5618,Município,Nova Iguaçu,3303500,RJ,None,Arquivado,17944.000156/2010-13,2010-12-03T00:00:00Z,Operação contratual interna,PAC - Saneamento - 2009 A,Instituição Financeira Nacional,Caixa Econômica Federal,Real,6.073730e+07,0,0,15/12/2010
4,5275,Município,Rio de Janeiro,3304557,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000075/2014-47,2014-04-01T00:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,2.715389e+09,1,0,29/05/2014


In [3]:
# 2) Seu código aqui
status_frequencia = df['status'].value_counts()
print("Os três status mais frequentes são:")
print(status_frequencia.head(3))

Os três status mais frequentes são:
status
Deferido                          106
Arquivado                          53
Arquivado por decurso de prazo     31
Name: count, dtype: int64


In [5]:
# 3) Seu código aqui
# Transformando em string e fatiando (slice) os últimos 4 caracteres
df['ano_status'] = df['data_status'].str[-4:]

# Alternativa usando split (separando pela barra e pegando o terceiro elemento)
# df['ano_status'] = df['data_status'].str.split('/').str[2]

df[['data_status', 'ano_status']].head()

frequencia_ano = df['ano_status'].value_counts()
print("Frequência de solicitações por ano:")
print(frequencia_ano)

Frequência de solicitações por ano:
ano_status
2008    37
2007    29
2023    28
2025    23
2014    20
2010    19
2019    16
2013    15
2020    14
2012    13
2022    11
2011    10
2009     9
2015     8
2021     5
2017     5
2016     4
2018     4
2002     4
2006     4
2026     3
2024     3
2004     1
2003     1
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [6]:
#1) Seu código aqui
def consultar_pvl(uf, tipo_interessado):
    # Formata a URL com os parâmetros desejados
    url = f"https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}"
    
    # Realiza a requisição
    r = requests.get(url)
    data_json = r.json()
    
    # Transforma em DataFrame
    df = pd.DataFrame(data_json['items'])
    return df

In [7]:
# 2) Seu código aqui
# 1. Busca os dados
df_mg = consultar_pvl(uf='MG', tipo_interessado='Estado')

# 2. Filtra pelo status solicitado
status_alvo = 'Arquivado por decurso de prazo'
quantidade = df_mg[df_mg['status'] == status_alvo].shape[0]

print(f"Quantidade de solicitações arquivadas por decurso de prazo em MG: {quantidade}")

Quantidade de solicitações arquivadas por decurso de prazo em MG: 1


In [8]:
# 3) Seu código aqui
# 1. Busca os dados de municípios da Bahia
df_ba_mun = consultar_pvl(uf='BA', tipo_interessado='Município')

# 2. Filtra apenas os deferidos
deferidos_ba = df_ba_mun[df_ba_mun['status'] == 'Deferido']

# 3. Conta ocorrências por município (coluna 'interessado')
municipio_top = deferidos_ba['interessado'].value_counts().idxmax()
total_top = deferidos_ba['interessado'].value_counts().max()

print(f"O município da Bahia com mais solicitações deferidas é {municipio_top} com {total_top} ocorrências.")

O município da Bahia com mais solicitações deferidas é Luís Eduardo Magalhães com 16 ocorrências.


In [9]:
# 4) Seu código aqui
# 1. Busca os dados do Estado da Bahia
df_ba_estado = consultar_pvl(uf='BA', tipo_interessado='Estado')

# 2. Salva o arquivo CSV
df_ba_estado.to_csv('solicitacoes_bahia_estado.csv', index=False, sep=';', encoding='utf-8-sig')

print("Arquivo 'solicitacoes_bahia_estado.csv' salvo com sucesso.")

Arquivo 'solicitacoes_bahia_estado.csv' salvo com sucesso.
